In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
# from torchvision import transforms
import os
import glob
import torch

In [2]:
df = pd.read_excel("../India_95/India.xlsx",0)

In [3]:
df.head()

,Number,Hgb,Gender,Age
0,1,12.2,0,29
1,2,8.0,1,36
2,3,10.7,1,30
3,4,8.3,1,39
4,5,7.8,1,29


In [4]:
from torchvision.models import resnet18, ResNet18_Weights

resnet = resnet18(weights=ResNet18_Weights.DEFAULT)
# or any of these variants
# from torchvision.models import resnet34, ResNet34_Weights
# from torchvision.models import resnet50, ResNet50_Weights
# from torchvision.models import resnet101, ResNet101_Weights
# from torchvision.models import resnet152, ResNet152_Weights

modules = list(resnet.children())[:-1]
resnet = nn.Sequential(*modules)

resnet.eval()

def preprocess_image(filename):
    input_image = Image.open(filename)

    input_image.load()
    background = Image.new("RGB", input_image.size, (255, 255, 255))
    background.paste(input_image, mask = input_image.split()[3])

    #background = input_image.convert('RGB')

    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    return preprocess(background).unsqueeze(0)

In [5]:
training = "../India_95/training"
testing = "../India_95/testing"
all_data = "../India_95/All"


training_img = glob.glob(f"{training}/*.png")
testing_img = glob.glob(f"{testing}/*.png")
all_img = sorted(glob.glob(f"{all_data}/*.png"))

In [6]:
All_Data = []
errors = []

for i in all_img:

    number = i.split("/")[-1].split(".")[0]
    label = df.loc[df['Number'] == int(number)]['Hgb'].tolist()[0]

    try:
        input_batch = preprocess_image(i)

        if torch.cuda.is_available():
            input_batch = input_batch.to('cuda')
            resnet.to('cuda')

        with torch.no_grad():
            output = resnet(input_batch)

        # data = [number,label]
        output = output.cpu()
        data = output.squeeze().numpy()
        data = data.flatten().tolist()

        data.insert(0,number)
        data.append(label)

        All_Data.append(data)

    except Exception as e:
        error = [number,e]
        errors.append(error)

In [7]:
headers = ['number']
for i in range(512):
    col = 'data_' + str(i)
    headers.append(col)
headers.append('label')
df_save = pd.DataFrame(All_Data, columns=headers)
df_save.to_csv('../India_95/Data.csv', index=False)

In [8]:
header2 = ['number','error_message']
df_error = pd.DataFrame(errors, columns=header2)
df_error.to_csv('../India_95/Error_files.csv', index=False)

In [9]:
len(All_Data)

95

In [10]:
len(errors)

0

In [11]:
import pycaret

In [12]:
data = pd.read_csv('../India_95/Data.csv')

In [13]:
data.head()

,number,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,...,data_503,data_504,data_505,data_506,data_507,data_508,data_509,data_510,data_511,label
0,1,1.902710,0.199951,0.000000,1.262051,0.903889,0.543976,0.899842,1.560050,1.567343,...,0.273968,0.155676,0.763023,0.112016,0.394547,2.553261,0.734258,0.070343,3.713481,12.2
1,10,2.293333,0.664260,0.009955,1.476898,0.112417,1.328928,0.967346,1.925749,0.870368,...,0.174050,0.119019,0.060720,0.429582,0.001138,1.609404,0.631039,0.507444,3.513334,11.3
2,11,1.183855,0.181115,0.000000,0.320417,0.086048,0.746380,0.331131,0.643436,1.909234,...,0.225710,0.264820,0.400918,0.982643,0.423948,2.026551,0.128508,0.333977,4.470862,13.2
3,12,0.931781,0.911379,0.008915,0.572634,0.294028,0.288136,0.273135,0.855321,1.187453,...,0.320516,0.117238,0.293577,0.415986,0.979531,1.763458,0.563302,0.472163,3.754123,10.6
4,13,1.601001,0.523836,0.023540,0.942168,0.495031,1.838428,0.274862,0.922636,1.848487,...,0.071390,0.097011,0.425649,0.196208,0.020918,1.190770,0.732096,0.243821,3.458430,10.6


In [14]:
data.describe(include="all")

,number,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,...,data_503,data_504,data_505,data_506,data_507,data_508,data_509,data_510,data_511,label
count,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,...,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000
mean,48.000000,1.547405,0.442150,0.022433,0.886328,0.329196,0.636073,0.386642,0.841855,1.019653,...,0.177360,0.136585,0.341165,0.411503,0.211359,1.984059,0.470011,0.416257,3.858305,11.471579
std,27.568098,0.455825,0.250682,0.054851,0.505611,0.218167,0.399620,0.275127,0.488153,0.451624,...,0.147612,0.160466,0.274337,0.221681,0.220928,0.408852,0.249992,0.269972,0.646042,2.075678
min,1.000000,0.787621,0.030672,0.000000,0.055270,0.003417,0.081934,0.011092,0.025567,0.238889,...,0.010760,0.000000,0.019363,0.064538,0.000000,0.946962,0.000000,0.005674,2.467052,7.600000
25%,24.500000,1.242174,0.210494,0.000000,0.464752,0.152370,0.337190,0.194679,0.512779,0.676136,...,0.066752,0.008231,0.119340,0.227603,0.054442,1.724594,0.265104,0.245242,3.430549,9.950000
50%,48.000000,1.441037,0.391682,0.000000,0.886322,0.294028,0.543976,0.335986,0.756196,1.016054,...,0.158331,0.061755,0.285028,0.372788,0.111555,2.024532,0.476418,0.358269,3.896665,11.300000
75%,71.500000,1.845794,0.645008,0.015342,1.162148,0.471432,0.857418,0.520012,1.143536,1.412772,...,0.228537,0.250332,0.510907,0.500783,0.314685,2.274940,0.617964,0.505025,4.270353,12.800000
max,95.000000,2.838402,1.003406,0.297709,2.511008,0.903889,1.887160,1.392577,2.243079,1.977049,...,0.690578,0.623085,1.026618,1.022975,0.979531,2.845803,1.224976,1.274765,5.282394,17.100000


In [15]:
from pycaret.regression import *
s = setup(data, target='label', ignore_features=['number'], 
          #log_experiment=True, experiment_name="resnet_2", log_data=True, 
          pca=True, pca_method="linear", 
          remove_multicollinearity=True, multicollinearity_threshold=0.5, 
          #feature_selection=True, feature_selection_method="sequential", n_features_to_select='auto', 
          normalize=True, 
          session_id=123)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Regression
3,Original data shape,"(95, 514)"
4,Transformed data shape,"(95, 67)"
5,Transformed train set shape,"(66, 67)"
6,Transformed test set shape,"(29, 67)"
7,Ignore features,1
8,Numeric features,512
9,Preprocess,True


In [16]:
best_r = compare_models(sort="RMSE")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,1.7519,4.5119,2.0630,-0.2100,0.1692,0.1605,0.1340
llar,Lasso Least Angle Regression,1.7519,4.5119,2.0630,-0.2100,0.1692,0.1605,0.0280
et,Extra Trees Regressor,1.7316,4.6498,2.0713,-0.1730,0.1696,0.1583,0.0390
dummy,Dummy Regressor,1.7746,4.5818,2.0761,-0.1992,0.1700,0.1622,0.0270
rf,Random Forest Regressor,1.7444,4.6301,2.0838,-0.2327,0.1708,0.1593,0.0480
en,Elastic Net,1.7552,4.6142,2.0884,-0.2506,0.1708,0.1608,0.0300
ada,AdaBoost Regressor,1.7321,4.8251,2.1257,-0.2939,0.1729,0.1577,0.0330
knn,K Neighbors Regressor,1.8346,4.9927,2.1617,-0.4187,0.1751,0.1656,0.0300
lightgbm,Light Gradient Boosting Machine,1.8604,4.9488,2.1835,-0.5420,0.1791,0.1687,0.0320
gbr,Gradient Boosting Regressor,1.8131,5.1764,2.2180,-0.5098,0.1814,0.1634,0.0360


In [17]:
evaluate_model(best_r)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [18]:
tuned_model = tune_model(best_r, fold = 10, n_iter = 500)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.8676,1.0791,1.0388,-0.3502,0.0885,0.0836
1,1.6608,3.8199,1.9545,-0.3629,0.1744,0.1762
2,1.6656,4.4097,2.0999,-0.0722,0.1909,0.1789
3,2.6361,8.9316,2.9886,-0.0245,0.2333,0.2279
4,1.7355,4.1025,2.0255,0.0097,0.1734,0.1662
5,1.7097,3.9499,1.9874,-0.3938,0.1734,0.1755
6,1.2667,2.1558,1.4683,-0.1750,0.1128,0.1035
7,2.1333,7.0454,2.6543,-0.0580,0.2050,0.1787
8,2.0144,5.3816,2.3198,-0.5178,0.1736,0.1556


Fitting 10 folds for each of 500 candidates, totalling 5000 fits


In [19]:
evaluate_model(tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [20]:
from pycaret.regression import *
s = setup(data, target='label', ignore_features=['number'], 
          #log_experiment=True, experiment_name="resnet_2", log_data=True, 
          #pca=True, pca_method="linear", 
          #remove_multicollinearity=True, multicollinearity_threshold=0.5, 
          #feature_selection=True, feature_selection_method="sequential", n_features_to_select='auto', 
          #normalize=True, 
          session_id=123)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Regression
3,Original data shape,"(95, 514)"
4,Transformed data shape,"(95, 513)"
5,Transformed train set shape,"(66, 513)"
6,Transformed test set shape,"(29, 513)"
7,Ignore features,1
8,Numeric features,512
9,Preprocess,True


In [21]:
best_r_base = compare_models(sort="RMSE")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,1.6473,4.3013,1.9895,-0.1650,0.1624,0.1503,0.0060
br,Bayesian Ridge,1.7249,4.3970,2.0439,-0.2067,0.1678,0.1581,0.0050
llar,Lasso Least Angle Regression,1.7746,4.5818,2.0761,-0.1992,0.1700,0.1622,0.0050
dummy,Dummy Regressor,1.7746,4.5818,2.0761,-0.1992,0.1700,0.1622,0.0040
lasso,Lasso Regression,1.7746,4.5818,2.0761,-0.1992,0.1700,0.1622,0.0040
en,Elastic Net,1.7769,4.5910,2.0805,-0.2096,0.1704,0.1625,0.0050
rf,Random Forest Regressor,1.7736,4.8455,2.1508,-0.3613,0.1757,0.1626,0.0380
huber,Huber Regressor,1.8569,4.8179,2.1713,-0.5782,0.1775,0.1662,0.0090
et,Extra Trees Regressor,1.8502,5.0762,2.1851,-0.3674,0.1778,0.1685,0.0290
ridge,Ridge Regression,1.8710,4.9585,2.1980,-0.5806,0.1791,0.1673,0.0050


In [22]:
from pycaret.regression import *
s = setup(data, target='label', ignore_features=['number'], 
          #log_experiment=True, experiment_name="resnet_2", log_data=True, 
          pca=True, pca_method="linear", 
          #remove_multicollinearity=True, multicollinearity_threshold=0.5, 
          #feature_selection=True, feature_selection_method="sequential", n_features_to_select='auto', 
          #normalize=True, 
          session_id=123)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Regression
3,Original data shape,"(95, 514)"
4,Transformed data shape,"(95, 67)"
5,Transformed train set shape,"(66, 67)"
6,Transformed test set shape,"(29, 67)"
7,Ignore features,1
8,Numeric features,512
9,Preprocess,True


In [23]:
best_r_base = compare_models(sort="RMSE")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,1.6473,4.3013,1.9895,-0.1650,0.1624,0.1503,0.0050
lightgbm,Light Gradient Boosting Machine,1.7429,4.4852,2.0216,-0.1766,0.1646,0.1588,0.0100
br,Bayesian Ridge,1.7247,4.3964,2.0438,-0.2066,0.1678,0.1580,0.0060
et,Extra Trees Regressor,1.7607,4.5669,2.0718,-0.2416,0.1689,0.1588,0.0190
dummy,Dummy Regressor,1.7746,4.5818,2.0761,-0.1992,0.1700,0.1622,0.0050
en,Elastic Net,1.7800,4.6076,2.0893,-0.2519,0.1715,0.1635,0.0070
lasso,Lasso Regression,1.7916,4.6360,2.0902,-0.2219,0.1713,0.1642,0.0060
llar,Lasso Least Angle Regression,1.7916,4.6360,2.0902,-0.2219,0.1713,0.1642,0.0060
rf,Random Forest Regressor,1.7574,4.7832,2.1063,-0.2207,0.1722,0.1593,0.0210
ada,AdaBoost Regressor,1.7766,4.7949,2.1117,-0.3931,0.1708,0.1573,0.0130
